In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')

In [3]:
import asyncio
import logging

from app.redis_transcribe.connection import get_redis_client
from app.settings import settings
from app.services.transcription.processor import Processor




redis_client = await get_redis_client(settings.redis_host, settings.redis_port,settings.redis_password)

processor = Processor(redis_client,max_length=30)



ModuleNotFoundError: No module named 'app.services.engine_api'

In [20]:
from app.services.audio.redis_models import Transcript

In [21]:
import pandas as pd
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = 1000

In [22]:
from app.services.audio.redis_models import Connection, Meeting, Transcriber

In [ ]:
await Transcript.get_all_transcripts_json(redis_client)

2025-02-20 12:38:24,869 - INFO - Successfully wrote transcripts to segments.json


'{\n  "meetings": {\n    "the-zdjv-byg": [\n      {\n        "content": " \\u044f \\u0434\\u0443\\u043c\\u0430\\u043b \\u0442\\u044b \\u0447\\u0442\\u043e-\\u0442\\u043e \\u043f\\u0440\\u043e \\u043d\\u0443 \\u0442\\u0438\\u043f\\u0430 \\u043a\\u0430\\u043a \\u0445\\u0440\\u0430\\u043d\\u0438\\u0442\\u044c \\u0442\\u0430\\u043c \\u0438 \\u0442\\u0430\\u043a \\u0434\\u0430\\u043b\\u0435\\u0435 \\u043d\\u043e \\u043e\\u043a\\u0435\\u0439 \\u0434\\u0430 \\u043d\\u0443\\u0436\\u043d\\u043e \\u043d\\u0443\\u0436\\u043d\\u043e \\u0437\\u0430\\u043f\\u0438\\u0441\\u0430\\u0442\\u044c",\n        "start_timestamp": "2025-02-13 15:32:16.430000+00:00",\n        "end_timestamp": "2025-02-13 15:32:30.750000+00:00",\n        "speaker": "Sergey Ryabenko",\n        "confidence": 0.09776536312849161,\n        "segment_id": 352,\n        "meeting_id": "the-zdjv-byg",\n        "words": [\n          [\n            " \\u044f",\n            0.43,\n            1.45\n          ],\n          [\n            " \

In [ ]:
df = await Transcript.get_all_transcripts_df(redis_client)
print(len(df))
df.drop(columns=['words']).tail(10)

33


,content,start_timestamp,end_timestamp,speaker,confidence,segment_id,meeting_id,server_timestamp,transcription_timestamp,present_user_ids,partially_present_user_ids
24,"а просто он будет, если домашний сервер занят,",2025-02-13 15:34:09.080000+00:00,2025-02-13 15:34:13.100000+00:00,None,0.000000,375,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-20 12:37:03.408791+00:00,[],[]
25,"а там только модельки крутятся больше,",2025-02-13 15:34:13.100000+00:00,2025-02-13 15:34:15.740000+00:00,None,0.000000,376,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-20 12:37:03.408791+00:00,[],[]
0,ничего вот тогда он будет поднимать инстансов google и будет скелете во сколько надо ну,2025-02-13 15:34:15.906000+00:00,2025-02-13 15:34:27.306000+00:00,Dmitriy Grankin,0.446842,377,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-20 12:38:04.278308+00:00,[],[00000000-0000-4000-a000-000000000001]
1,прям серьезно вообще ну а ну просто как бы заверну то все то есть есть просто сервера,2025-02-13 15:34:29.386000+00:00,2025-02-13 15:34:37.666000+00:00,None,0.000000,378,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-20 12:38:04.278308+00:00,[],[]
2,сервис который говорит дай модельку я тебе дам а пешку и там буду все авто скилить,2025-02-13 15:34:37.666000+00:00,2025-02-13 15:34:43.026000+00:00,None,0.000000,379,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-20 12:38:04.278308+00:00,[],[]
3,я завернул одно в другое за open source а там дальше просто обычная очередь если там,2025-02-13 15:34:44.626000+00:00,2025-02-13 15:34:52.786000+00:00,None,0.000000,380,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-20 12:38:04.278308+00:00,[],[]
4,занят следующую жену круто здорово если все это очень быстро поднимается с точки,2025-02-13 15:34:52.786000+00:00,2025-02-13 15:35:03.036000+00:00,None,0.000000,381,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-20 12:38:04.278308+00:00,[],[]
5,зрения хранения данных это как получается как потом по всем этим инстансом бегать искать,2025-02-13 15:35:03.036000+00:00,2025-02-13 15:35:08.856000+00:00,None,0.000000,382,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-20 12:38:04.278308+00:00,[],[]
6,нет данные в одном месте все хранятся,2025-02-13 15:35:08.856000+00:00,2025-02-13 15:35:14.436000+00:00,None,0.000000,383,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-20 12:38:04.278308+00:00,[],[]
7,только модельки,2025-02-13 15:35:14.656000+00:00,2025-02-13 15:35:15.636000+00:00,None,0.000000,384,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-20 12:38:04.278308+00:00,[],[]
